# Analyzer Notebook
## Introduction
This notebook contains code and graphs intended for exploratory analysis over the data and how the current `tix-time-processor` handles it. By no means should it be used as a testing method for the code, only to understand how the analysis are made and to improve further the results.

### Disclaimer 
We understand that having a Python Notebook inside a functioning service is wrong. This should be in another repository, with its own versioning and such. But such is life... This will be left like this for the moment until someone with more time can fix it. Kind regards.

### How to use this notebook
For the moment, the notebook should live in the root directory of the sourcecode repository such as it is now. Also, there should be a directory next to this by the name of `batch-test-reports`, which it should contain the data to analyze. It is **strongly** recommended that the data is split among directories containing the expected processing input for an specific moment in time. This is to better understand the processing.

## The Analysis
We will first setup everything in order to then run the code. Once the code was run, we will analyze with graphics how is it that we got to the show result
### The setup

In [17]:
# This will be a matplotlib notebook
%matplotlib notebook
# We import all the dependencies here
from functools import partial
from os import getcwd
from os.path import join

import pprint
import matplotlib.pyplot as plt
import numpy as np
from processor import analysis, reports
import pandas as pd

# And setup the basic stuff, configure the pretty printer, load the reports, setup the analyzer and
# define some more stuff.
pprinter = pprint.PrettyPrinter(indent=2)
reports_directory = join(getcwd(), 'batch-test-reports', '1511633889')
reports_handler = reports.ReportHandler(reports_directory)
ip, obs_set = reports_handler.get_ip_and_processable_observations()
if ip is None and obs_set is None:
    raise ValueError('No enough processable reports in directory {}'.format(reports_directory))
analyzer = analysis.Analyzer(obs_set)

FileNotFoundError: [Errno 2] No such file or directory: '/home/fmartinez/workspaces/personal/tix-time-processor/batch-test-reports/1511633889/failed-results'

In [6]:
def get_bins_and_bins_probabilities_dfs(histogram, probabilities_name='probabilities'):
    bins_df = pd.DataFrame([{'min_value': bin_.min_value,
                             'mid_value': bin_.mid_value,
                             'max_value': bin_.max_value,
                             'width': bin_.width,
                             'data_points_qty': len(bin_.data)}
                            for bin_ in histogram.bins])
    bins_probabilities_normalized_df = pd.DataFrame(histogram.bins_probabilities,
                                                    columns=[probabilities_name]) * (10 ** 8)
    bins_probabilities_df = pd.concat([bins_probabilities_normalized_df, 
                                       bins_df['mid_value']],
                                      axis=1)
    return bins_df, bins_probabilities_df

def plot_histogram_and_pdf(fig, axs, fixed_size_bin_histogram, histogram_name, logy=True, logx=True):
    bins_df, bins_probabilities_df = get_bins_and_bins_probabilities_dfs(fixed_size_bin_histogram, histogram_name)
    data = [fixed_size_bin_histogram.characterization_function(d) 
            for d in fixed_size_bin_histogram.data]
    ranges = [b.min_value for b in fixed_size_bin_histogram.bins]
    ranges.append(fixed_size_bin_histogram.bins[-1].max_value)
    axs[0].hist(data, bins=ranges, edgecolor='black')
    # upstream_bins_df.plot.bar('mid_value', 'width', logy=True, ax=axs[0])
    bins_probabilities_df.plot('mid_value', histogram_name, 
                               logy=True, 
                               logx=True, 
                               ax=axs[1])
    axs[1].axvline(fixed_size_bin_histogram.mode, color='green', label='mode')
    axs[1].axvline(fixed_size_bin_histogram.threshold, color='red', label='threshold')
    axs[1].legend()

In [ ]:
# Finally we print the results... This is what is expected to go to the API
pprinter.pprint(analyzer.get_results())

### The RTT Histogram and the clock correction

In [ ]:
fig, axs = plt.subplots(1, 2)
plot_histogram_and_pdf(fig, axs, analyzer.rtt_histogram, 'rtt_probabilities')

In [ ]:
{'tau': analyzer.rtt_histogram.mode, 'tau_threshold': analyzer.rtt_histogram.threshold}

### Clock Fixer

In [ ]:
# phis_timestamps, phis = tuple(zip(*analyzer.clock_fixer.phis_for_minute))
fig, ax = plt.subplots()
# regression_values = [analyzer.clock_fixer.slope * ts + analyzer.clock_fixer.intercept for ts in phis_timestamps]

# ax.plot(phis_timestamps, regression_values, '-', color='red', label='regression')
# ax.legend()
obs_timestamps, rtts = tuple(zip(*[(o.day_timestamp, analysis.observation_rtt_key_function(o)) 
                                   for o in analyzer.clock_fixer.observations]))
ax.scatter(obs_timestamps, rtts, s=1)

In [ ]:
fig, ax = plt.subplots()
obs_by_day_timestamp = sorted(analyzer.observations, key=lambda o: o.day_timestamp)
y, x = tuple(zip(*[(analyzer.clock_fixer.phi_function(o.day_timestamp), o.day_timestamp)
                   for o in obs_by_day_timestamp]))

ax.plot(x, y)

In [ ]:
rtts = [analysis.observation_rtt_key_function(o) for o in analyzer.clock_fixer.observations]
max_rtt = max(rtts)
min_rtt = min(rtts)
min_rtt/10**9, max_rtt/10**9

In [ ]:
# {'slope': analyzer.clock_fixer.slope, 'intercept': analyzer.clock_fixer.intercept}

In [ ]:
# {'r_value': analyzer.clock_fixer.r_value, 'p_value': analyzer.clock_fixer.p_value, 'std_err': analyzer.clock_fixer.std_err}

### Data Usage

#### Upstream

In [ ]:
fig, axs = plt.subplots(1, 2)
plot_histogram_and_pdf(fig, axs, analyzer.usage_calculator.upstream_histogram, 'upstream_probabilities')

In [ ]:
{'upstream mode': analyzer.usage_calculator.upstream_histogram.mode,
 'upstream threshold': analyzer.usage_calculator.upstream_histogram.threshold}

#### Dowstream

In [ ]:
fig, axs = plt.subplots(1, 2)
plot_histogram_and_pdf(fig, axs, analyzer.usage_calculator.downstream_histogram, 'downstream_probabilities')

In [ ]:
{'downstream mode': analyzer.usage_calculator.downstream_histogram.mode,
 'downstream threshold': analyzer.usage_calculator.downstream_histogram.threshold}

### Quality and Hurst values
#### Quality